# Regression-1: Build quick models

ここではモデル構築の大きな流れを掴みましょう。まずは、サンプルデータ（回帰用）を読み込みます。

In [1]:
# import the data for regression
import pandas as pd
from IPython.core.display import display
from sklearn.datasets import load_boston
dataset = load_boston()

# set dataframe
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

# check the shape
print('--------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('--------------------------------------')
print(y.describe())
display(X.join(y).head())

--------------------------------------
X shape: (506,13)
y shape: (506,1)
--------------------------------------
                y
count  506.000000
mean    22.532806
std      9.197104
min      5.000000
25%     17.025000
50%     21.200000
75%     25.000000
max     50.000000


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,y
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


アンサンブル学習（ブースティング）の一つである勾配ブースティンレグレッサーを読み込み学習（fit）します。予測値と正解データの整合性をR2値で評価しています。データを読み込み、学習させ、評価する流れを掴んで下さい。

In [ ]:
# import libraries
from sklearn.ensemble import GradientBoostingRegressor

# build the model
est = GradientBoostingRegressor(n_estimators=100,
                                max_depth=3,
                                random_state=0)
est.fit(X, y.as_matrix().ravel())

# check the model performance by R2 socre
from sklearn.metrics import r2_score
y_true = y.as_matrix().ravel()
y_pred = est.predict(X)
r2 = r2_score(y_true, y_pred)
print('R2 score: %.3f' % r2)

モデル構築は以上ですが、予測を目的とした場合、モデル構築のステップは増加します。それは未知データへの当てはまりに興味があるためです。予測上の最大の敵は<b>過学習</b>であり、この過学習へ対処するためのステップが増えると理解して下さい。<b>交差検証</b>はその対処法の一つで、汎化性能を確認しベストなモデルを選択するのに役立ちます。

In [ ]:
# import libraries
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 交差検証のためデータを訓練とテストに分割
# 訓練を80%,テストを20%に分割
# 訓練とテストにランダム分割するだけの単純な交差検証は, hold-outと呼ばれる
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.20,
                                                 random_state=1)
# 整形
y_train = y_train.as_matrix().ravel()
y_test = y_test.as_matrix().ravel()

# 二つのモデルを構築(比較用)
# Standard model
est1 = GradientBoostingRegressor(max_depth=3,random_state=1)
est1.fit(X_train,y_train)

# Complex model
est2 = GradientBoostingRegressor(max_depth=10,random_state=1) 
est2.fit(X_train,y_train)

# モデルパフォーマンス指標(R2)の取得
# for training data
r2_est1_train = r2_score(y_train,est1.predict(X_train))
r2_est2_train = r2_score(y_train,est2.predict(X_train))

# for test data
r2_est1_test = r2_score(y_test,est1.predict(X_test))
r2_est2_test = r2_score(y_test,est2.predict(X_test))

# 性能指標の表示
print('Train Score(est1, est2):(%.3f, %.3f)'% (r2_est1_train, r2_est2_train))
print('Test Score(est1, est2):(%.3f, %.3f)'% (r2_est1_test, r2_est2_test))

# est1のパラメータ条件で最終モデルを構築
# est1.fit(X, y.as_matrix().ravel())
# print('----------------------------------------')
# print(est1)